In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from src.augmentations import CustomAugmentations

import numpy as np
import random
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

import os
os.environ['PYTHONHASHSEED'] = '42'


keras.utils.set_random_seed(42)
# tf.config.experimental.enable_op_determinism()

In [2]:
height,width=32,32
batch_size=64

In [3]:
from src.augmentations.CustomAugmentations import CustomAugmentationsTF

# Define the augmentation probabilities
p_dict = {
  # "flip": 0.5,
  # "transpose": 0.5,
  # "gauss_noise": 0.2,
  # "brightness_contrast": 0.5,
  "hue_saturation_value": 0.5,
}

p_dicts = []
for aug_type in p_dict.keys():
    p_d = {aug_type: .5}
    p_dicts.append(p_d)

p_dict = {
  "flip": 0.5,
  "transpose": 0.5,
  "gauss_noise": 0.5,
  "brightness_contrast": 0.5,
  "hue_saturation_value": 0.5 ,
}
p_dicts.append(p_dict)

p_dicts


[{'hue_saturation_value': 0.5},
 {'flip': 0.5,
  'transpose': 0.5,
  'gauss_noise': 0.5,
  'brightness_contrast': 0.5,
  'hue_saturation_value': 0.5}]

In [4]:
import matplotlib.pyplot as plt

augmented = original_dataset.map(lambda image, label: (custom_aug.augment(image), label))



In [5]:
from copy import deepcopy
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm.notebook import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

from src.kerasCNN import *
import pickle


EPOCHS = 50

histories = list()
accuracies = list()
class_accuracies = list()
conf_mats = list()

for augs in tqdm(p_dicts, desc="Augmentation"):
    aug_str = "_".join([key + str(value) for key, value in augs.items()])
    print(f"Augmentations: {aug_str}")

    for iteration in tqdm(range(5), desc="Iteration"):
        train_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/train",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="training",
        )

        val_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/valid",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="validation",
        )

        test_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/test",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=False,
        )

        dnn_model = create_cnn(
            kernel_sizes=[3, 3, 3,],
            num_filters=[32, 64, 64],
            fc_sizes=[128, 64, 64],
        )
       

        history = {
            "loss": list(),
            "accuracy": list(),
            "val_loss": list(),
            "val_accuracy": list(),
        }

        print(f"Fitting model {iteration}")
        for epoch in tqdm(range(EPOCHS), desc="Epoch"):
            print(f"Epoch {epoch + 1}/{EPOCHS}")
            custom_aug = CustomAugmentationsTF(augs)
            train_set_aug = train_set.map(lambda x, y: (custom_aug.augment(x), y))

            hist_ = dnn_model.fit(train_set_aug, validation_data=val_set, epochs=1)
            history["loss"].append(hist_.history["loss"])
            history["accuracy"].append(hist_.history["accuracy"])
            history["val_loss"].append(hist_.history["val_loss"])
            history["val_accuracy"].append(hist_.history["val_accuracy"])

        print(f"Predicting model {iteration}")
        preds = dnn_model.predict(test_set, verbose=2)
        preds = preds.argmax(axis=1)
        classes = test_set.class_names
        test_labels = list()
        for images, labels in test_set:
            class_labels = [int(label) for label in labels]
            test_labels.extend(class_labels)
        test_labels = np.array(test_labels)

        conf_mat = confusion_matrix(test_labels, preds)
        accuracy = accuracy_score(test_labels, preds)
        print(f"Accuracy: {accuracy} for model {iteration}")
        class_accuracy = conf_mat.diagonal() / conf_mat.sum(axis=1)

        print(f"Saving model {iteration}")
        dnn_model.save(f"../models/BESTClassicCNN_aug_{aug_str}_{iteration}.h5")
        histories.append(deepcopy(history))
        accuracies.append(deepcopy(accuracy))
        class_accuracies.append(deepcopy(class_accuracy))
        conf_mats.append(deepcopy(conf_mat))


    with open(f"results/HISTORY_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(histories, f)

    with open(f"results/ACCURACY_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(accuracies, f)

    with open(f"results/CLASS_ACCURACY_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(class_accuracies, f)

    with open(f"results/CONF_MAT_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(conf_mats, f)

Augmentation:   0%|          | 0/2 [00:00<?, ?it/s]

Augmentations: hue_saturation_value0.5


Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Found 90000 files belonging to 10 classes.
Using 85500 files for training.
Found 90000 files belonging to 10 classes.
Using 4500 files for validation.
Found 90000 files belonging to 10 classes.
Fitting model 0


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/50
1336/1336 [==============================] - 29s 21ms/step - loss: 1.6092 - accuracy: 0.4085 - val_loss: 2.2670 - val_accuracy: 0.2264
Epoch 2/50
1336/1336 [==============================] - 29s 21ms/step - loss: 1.5439 - accuracy: 0.4453 - val_loss: 9.5429 - val_accuracy: 0.1222
Epoch 3/50
1336/1336 [==============================] - 29s 21ms/step - loss: 1.3738 - accuracy: 0.5065 - val_loss: 4.7761 - val_accuracy: 0.1213
Epoch 4/50
1336/1336 [==============================] - 27s 20ms/step - loss: 1.3285 - accuracy: 0.5254 - val_loss: 3.6870 - val_accuracy: 0.1622
Epoch 5/50
1336/1336 [==============================] - 28s 21ms/step - loss: 1.2984 - accuracy: 0.5375 - val_loss: 3.3717 - val_accuracy: 0.1796
Epoch 6/50
1336/1336 [==============================] - 28s 21ms/step - loss: 1.2084 - accuracy: 0.5676 - val_loss: 3.5092 - val_accuracy: 0.1616
Epoch 7/50
1336/1336 [==============================] - 28s 21ms/step - loss: 1.4795 - accuracy: 0.4612 - val_loss: 2.1769 -

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/50
1336/1336 [==============================] - 32s 23ms/step - loss: 1.6831 - accuracy: 0.3826 - val_loss: 2.4382 - val_accuracy: 0.2020
Epoch 2/50
1336/1336 [==============================] - 26s 19ms/step - loss: 1.4712 - accuracy: 0.4618 - val_loss: 1.6226 - val_accuracy: 0.4293
Epoch 3/50
1336/1336 [==============================] - 29s 22ms/step - loss: 1.4855 - accuracy: 0.4628 - val_loss: 4.3138 - val_accuracy: 0.1873
Epoch 4/50
1336/1336 [==============================] - 28s 21ms/step - loss: 1.3315 - accuracy: 0.5183 - val_loss: 1.2664 - val_accuracy: 0.5384
Epoch 5/50
1336/1336 [==============================] - 30s 23ms/step - loss: 1.4416 - accuracy: 0.4825 - val_loss: 6.8095 - val_accuracy: 0.1849
Epoch 6/50
1336/1336 [==============================] - 26s 19ms/step - loss: 1.2737 - accuracy: 0.5434 - val_loss: 1.2250 - val_accuracy: 0.5529
Epoch 7/50
1336/1336 [==============================] - 29s 22ms/step - loss: 1.3169 - accuracy: 0.5309 - val_loss: 5.1979 -

KeyboardInterrupt: 